import numpy as np

from music_constants import base_notes, base_intervals, catalog

class Scale:

    def __init__(self, tonic, intervals):
        self.base_notes = base_notes
        self.base_intervals = base_intervals
        self.catalog = catalog

        self.tonic = tonic[:1]
        self.accident = tonic[1:]
        self.notes = self.base_notes[self.base_notes.index(self.tonic):] + self.base_notes[:self.base_notes.index(self.tonic)]
        #print(self.notes)
        
        if isinstance(intervals, str):
            self.intervals = self.catalog[intervals.lower()]
        else:
            self.intervals = intervals

    def calculate_distances(self):
        semitones = np.array(self.semitones_from_tonic())
        #print("semitones:", semitones)
        distances = semitones[1:] - semitones[:-1]
        return distances

    def calculate_accidents_from_difference(self, note, difference):
        #print(note, difference)
        if difference > 0:
            return '#'*abs(difference)
        elif difference < 0:
            return 'b'*abs(difference)
        else:
            if "#" in note:
                return "#"
            if "b" in note:
                return "b"
            else:
                return ''

    def debug_mixed_accidents(self):
        self.notes = [element.replace('#b', '').replace('b#', '') for element in self.notes]

    def generate_scale(self):
        distances = self.calculate_distances()
        #print("distances:", distances)

        for i in range(len(self.notes) - 1):
            
            if np.isin(self.notes[i], ["E", "B"]):
                difference = distances[i] - 1
            else:
                difference = distances[i] - 2

            self.notes[i+1] += self.calculate_accidents_from_difference(self.notes[i], difference)
        
        self.notes = [note + self.accident for note in self.notes]
        self.debug_mixed_accidents()


    def semitones_from_tonic(self):
        semitones = []
        for interval in self.intervals:
            semitones.append(self.calculate_semitones(interval))
        return semitones
    
    def calculate_semitones(self, interval):
        
        interval_root = self.base_intervals[interval[-1]]
        accidents = interval[:-1]

        # Sum 1 for each # and substract 1 for each b in accidents
        for accident in accidents:
            if accident == '#':
                interval_root += 1
            elif accident == 'b':
                interval_root -= 1

        return interval_root

scale = Scale("Fb", "lydian b7")
scale.generate_scale()
print(scale.notes)

In [2]:
from scale import Scale
from key import Key

scale = Scale("C", "1 b2 b3 4 5 6 b7".split(" "))
key = Key(scale)

key.chords

['Cm7', 'Db7M(#5)', 'Eb7', 'F7', 'Gm7(b5)', 'Am7(b5)', 'Bbm7M']